In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#read table from the website
res = requests.get("https://photius.com/rankings/world2050_rank.html")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

#get country names
country_name = pd.Series(df[0][1])
#get 2000 population
population_2000 = pd.Series(df[0][3])
#get 2015 population
population_2015 = pd.Series(df[0][4])

#join 3 series created above into the dataframe
population_df = country_name.to_frame().join(population_2000).join(population_2015)

#clean up the dataframe
population_df.rename(columns={1:'Country',
                             3 : '2000',
                             4 : '2015'}, inplace=True)
population_df = population_df[6:233]

population_df['2015'] = population_df['2015'].astype(float)
population_df['2000'] = population_df['2000'].astype(float)

population_df['Delta'] = population_df['2015'] - population_df['2000']

population_df.set_index('Country', inplace=True)
print(population_df)

#record dataframe to csv
population_df.to_csv('Population.csv')


                                  2000       2015     Delta
Country                                                    
India                        1008937.0  1230484.0  221547.0
China                        1275133.0  1410217.0  135084.0
United States of America      283230.0   321225.0   37995.0
Pakistan                      141256.0   204267.0   63011.0
Indonesia                     212092.0   250068.0   37976.0
Nigeria                       113862.0   165313.0   51451.0
Bangladesh                    137439.0   183159.0   45720.0
Brazil                        170406.0   201393.0   30987.0
Dem. Rep. of the Congo         50948.0    84045.0   33097.0
Ethiopia                       62908.0    89765.0   26857.0
Mexico                         98872.0   119175.0   20303.0
Philippines                    75653.0    95881.0   20228.0
Vietnam                        78137.0    94413.0   16276.0
Iran (Islamic Republic of)     70330.0    87103.0   16773.0
Egypt                          67884.0  

In [2]:
#USA refugee
usa = "USA_cleaned_final.csv"
can = "can_cleaned_final.csv"
usa_df = pd.read_csv(usa)
can_df=pd.read_csv(can)

# Merge two dataframes using an inner join
merge_table = pd.merge(usa_df, can_df, on=["Origin","Year"])

merge_table = merge_table.iloc[:,[0,2,4,7]]

merge_table= merge_table.rename(columns={"Total Population_x": 'USA',
                                "Total Population_y": 'CAN'
                                })
merge_table

merge_table['Noth America'] = merge_table['USA'] - merge_table['CAN']

In [3]:
merge_table['Noth America'] = merge_table['USA'] + merge_table['CAN']
merge_table = merge_table.groupby(['Origin']).sum()
refudies_df = merge_table.drop(columns=['Year'])
refudies_df

merged = pd.concat([refudies_df, population_df], axis=1, join_axes=[refudies_df.index])

merged.rename(columns={'USA':'USA Refugies from 2000 to 2015',
                        'CAN' : 'CAN Refugies from 2000 to 2015',
                             'Noth America' : 'Total Noth America Regies 15 years',
                      '2000' : "Origin Country Population at 2000 x1000",
                      '2015' : "Origin Country Population at 2015 x1000",
                      'Delta' : 'Origin Country Population Change (15 years)'}, inplace=True)

merged

,USA Refugies from 2000 to 2015,CAN Refugies from 2000 to 2015,Total Noth America Regies 15 years,Origin Country Population at 2000 x1000,Origin Country Population at 2015 x1000,Origin Country Population Change (15 years)
Origin,,,,,,
Afghanistan,78694,125145,203839,21765.0,35577.0,13812.0
Albania,98106,34190,132296,3134.0,3439.0,305.0
Algeria,8925,25637,34562,30291.0,38022.0,7731.0
Angola,4833,13466,18299,13134.0,20795.0,7661.0
Antigua and Barbuda,51,661,712,65.0,68.0,3.0
Argentina,4355,8491,12846,37032.0,43498.0,6466.0
Armenia,101067,2913,103980,3787.0,3808.0,21.0
Aruba,0,0,0,101.0,157.0,56.0
Australia,37,100,137,NaN,NaN,NaN


In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
refudies_df.to_csv("refudies_df.csv", index=False, header=True)